In [39]:
'''
1. Fresh run on all genomes with the lastest GeneMark-ES
2. Parse GeneMark-ES output model file in the last run and plot exon/spacer/intron length distribution 
3. find connect between exon density and the exon/spacer/intron distribution
'''
import pandas as pd
fulldata = pd.read_csv("../1_DataSmall/fullTableInfoGff3GffRNA.csv")
fulldata.index = fulldata['shortName']

# The files under the fastapath were obtained from the script created in 2_Consolidation
# buildingMarkovModel part
fastaPath ="/home/richard/largeDataSet/genomeFasta/" 
targetConvergePath = "/home/richard/largeDataSet/ES_Run_Result_Aug27/"
targetPath = "/scratch/fungi/ES_Run_Aug2015/"

In [78]:
## make scripts to run GeneMark-ES that distribute the work to m machines


##########################################################################
# first of all, copy fasta files to nodes local drive: scratch
# command to check is nodes are good: for((i=10;i<34;i++)); do ssh node0"$i" echo "$i";done
# nodes 007, 031 fails
nodeNum = ["00"+str(i) for i in range(1,34) if i!=7 and i!=31]; nodeNum = map(lambda x: "node"+x[-3:], nodeNum)
print "Number of nodes active:",len(nodeNum)
numNode = len(nodeNum)

# split shortName column as a list of 31 sublist
shortName = list(fulldata['shortName'])
shortNameSublists = [shortName[i::numNode] for i in range(numNode)]

# assign each node a sublist of shortName
nodeJob = zip(nodeNum,shortNameSublists)

# make folder ES_Run_Aug2015 on each node's local drive, /scratch/fungi/ and copy the relavent shortName folder
# mkdir to nodes was performed on command line
script = ""
for node,shorNames in nodeJob:
    for shortName in shorNames:
        script += "ssh {} cp -r {}{} {}\n".format(node,fastaPath,shortName, targetPath)
with open("/home/richard/tempFile/cp_fasta.sh",'w') as f: f.write(script)
##########################################################################

##########################################################################
#execute GeneMark-ES on from each node

es_command = "/home/alexl/DISTR/gmes_petap/gmes_petap.pl --ES --fungus --min_contig 10000 --cores 8 --v  --seq "
script = ""
for node,shorNames in nodeJob:
    for shortName in shorNames:
        try:
            script += 'nohup ssh {} "cd {}{}; {}{}; cd .." &\n '.format(node,targetPath,shortName,es_command,fulldata.loc[shortName]['fastaName'][:-3])
        except:
            print "problem with",shortName
with open("/home/richard/tempFile/run_ES_Aug2015.sh",'w') as f: f.write(script)  

# print script
##########################################################################
## The above script has problem
## the correct way to do it is to add nohup the first time it use a new node

## copy mod file on the 4th run
modfilePath = "run/ES_C_4/ES_C_4.mod"
targetModPath = "/home/richard/largeDataSet/ES_Run_modfileAug2015/"
script = ""
for node,shorNames in nodeJob:
    for shortName in shorNames:
        script += 'ssh {} cp {}{}/{} {}{}.ES_C_4.mod\n'.format(node, targetPath, shortName,modfilePath,targetModPath, shortName)
with open("/home/richard/tempFile/cp_run_ES_mod_Aug2015.sh",'w') as f: f.write(script)  

Number of nodes active: 31
problem with Neute_matA2
problem with Pospl1
problem with Crypa2
problem with Ustma1
problem with Sporo1


In [ ]:
## create an extra column to label if ES is successful